In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import seaborn as sns
import random
import csv
import math
import pickle
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier


ModuleNotFoundError: No module named 'xgboost'

In [39]:
!pip install xgboost

     |████████████████████████████████| 193.7 MB 39 kB/s  eta 0:00:01    |██████▏                         | 37.4 MB 289 kB/s eta 0:09:00     |████████████▌                   | 75.6 MB 369 kB/s eta 0:05:20     |█████████████████               | 103.1 MB 262 kB/s eta 0:05:46     |███████████████████             | 115.5 MB 220 kB/s eta 0:05:55


In [2]:
df_train=pd.read_csv('final_train_features.csv')
df_test=pd.read_csv('final_test_features.csv')
print(df_train.shape)
print(df_test.shape)

(6342012, 27)
(1585504, 27)


In [4]:
df_train= df_train[df_train['link'].notna()]
df_test=df_test[df_test['link'].notna()]
print(df_train.shape)
print(df_test.shape)

(3171006, 27)
(792752, 27)


In [29]:
data=pd.concat([df_train,df_test])
data=data.sample(frac=0.13)
data.shape

(515289, 27)

In [5]:
data=pd.concat([df_train,df_test])
data=data.sample(frac=0.0139)
data.shape

(55096, 27)

In [6]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(data, test_size=0.2)
print(df_train.columns)

Index(['source_node', 'destination_node', 'link', 'num_followers_s',
       'num_followers_d', 'num_followees_s', 'num_followees_d',
       'inter_followers', 'inter_followees', 'jaccard_followee',
       'jaccard_followers', 'cosine_followers', 'shortest_path',
       'cosine_followees', 'adar_index', 'page_rank_s', 'page_rank_d',
       'katz_s', 'katz_d', 'hubs_s', 'hubs_d', 'authorities_s',
       'authorities_d', 'weight_in', 'weight_out', 'weight_total',
       'follows_back'],
      dtype='object')


In [7]:
X_train=df_train.drop(['source_node','destination_node','link'],axis=1)
y_train=df_train.link
X_test=df_test.drop(['source_node','destination_node','link'],axis=1)
y_test=df_test.link

In [32]:
parameter_grid = {'C': [ 10, 100,200],'gamma': [0.01,0.055, 0.001],'kernel':['rbf']}
model2_random=GridSearchCV(estimator =SVC(), param_grid = parameter_grid, 
                          cv =5, n_jobs = -1, verbose = 50,scoring='f1')        

In [33]:
model2_random.fit(X_train,y_train)
model2_random.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START C=10, gamma=0.01, kernel=rbf................................
[CV 1/5; 1/4] END C=10, gamma=0.01, kernel=rbf;, score=0.900 total time=127.0min
[CV 5/5; 4/4] START C=100, gamma=0.001, kernel=rbf..............................
[CV 5/5; 4/4] END C=100, gamma=0.001, kernel=rbf;, score=0.900 total time=134.3min
[CV 2/5; 4/4] START C=100, gamma=0.001, kernel=rbf..............................
[CV 2/5; 4/4] END C=100, gamma=0.001, kernel=rbf;, score=0.902 total time=135.2min
[CV 5/5; 2/4] START C=10, gamma=0.001, kernel=rbf...............................
[CV 5/5; 2/4] END C=10, gamma=0.001, kernel=rbf;, score=0.900 total time=135.3min
[CV 4/5; 1/4] START C=10, gamma=0.01, kernel=rbf................................
[CV 4/5; 1/4] END C=10, gamma=0.01, kernel=rbf;, score=0.900 total time=136.4min
[CV 3/5; 4/4] START C=100, gamma=0.001, kernel=rbf..............................
[CV 3/5; 4/4] END C=100, gamma=0.001, kernel

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [34]:
train_score = f1_score(y_train,model2_random.predict(X_train))
test_score = f1_score(y_test,model2_random.predict(X_test))
print("train f1-score is",train_score)
print("test f1-score is",test_score)

train f1-score is 0.9011470134272197
test f1-score is 0.8991926088762372
